In [1]:
#IMPORT lib(s)
import matplotlib.pyplot as plt
import random
import os
import matplotlib.image as mpimg
import cv2
import numpy as np
import math
from tensorflow.keras.models import load_model
from cvzone.HandTrackingModule import HandDetector
import time

In [2]:
model=load_model(r'C:\Users\Lenovo\nn_calculator_project\calculator7.h5')

In [3]:
#IMPORTANT
#function take img array, height , width
#to convert image of aspect ratio of 1:1.... height=80 and weight=80
def resize_img(img,imgh,imgw):
    imgsize=80
    #WHITE BACKGROUND
    imgwhite=np.ones((imgsize,imgsize,3),np.uint8)*255

    #if hight>width
    #adjusting the width
    if imgh>imgw:
        k=imgsize/imgh
        wcal=math.floor(k*imgw)
        #reshaping the hight to imgsize and width to calculated width
        imgresize=cv2.resize(img,(wcal,imgsize))
        #wgap is to bring the image in middle
        wgap=math.floor((imgsize-wcal)/2)
        #putting image over white background
        imgwhite[:,wgap:wcal+wgap]=imgresize


    #if hight<width
    #adjysting the hight
    else:
        k=imgsize/imgw
        hcal=math.floor(k*imgh)
        imgresize=cv2.resize(img,(imgsize,hcal))
        hgap=math.floor((imgsize-hcal)/2)
        imgwhite[hgap:hcal+hgap,:]=imgresize
    return imgwhite

In [4]:
def prediction(img):
    #resize the image (80,80,3)
    m=resize_img(img,len(img),len(img[0]))
    #normalization
    m=m/255
    #convert into (1,80,80,3)
    m=np.array([m])
    #predict the value in the form of array
    s=model.predict(m,verbose=0)
    #return the index of the maximum from numpy
    return np.argmax(s)

In [7]:
cap=cv2.VideoCapture(0)
detector=HandDetector(maxHands=1)

offset=20
imgsize=80
prev=time.time()
string=''
while True:
    success,img=cap.read()
    hands,img=detector.findHands(img)
    if hands:
        hand=hands[0]
        x,y,w,h=hand['bbox']
        
        imgCrop=img[y-offset:y+h+offset,x-offset:x+w+offset]
        
        cur=time.time()
        if cur-prev>3:
            prev=cur
            result=prediction(imgCrop)
            if len(string)!=0 and result==0:
                if string[len(string)-1] in ['+','-','*','/']:
                    pass
                else:
                    string+=str(result)
            elif string=='' and (result==0 or result==10 or result==11 or result==12 or result==13):
                string=''
            elif result==10:
                string+='+'
            elif result==11:
                string+='-'
            elif result==12:
                string+='*'
            elif result==13:
                string+='/'
            elif string=='' and result==15:
                string=''
            elif string!='' and result==15:  #DELETE
                string=string[:len(string)-1]
            elif string=='' and result==14:
                print("No output")
                break
            elif string!='' and result==14:
                valid=input("Enter 's' if you want to calculate or press any key to continue: ")
                if valid=='s':
                    print(eval(string))
                    break
                else:
                    pass
            else:
                string+=str(result)
            print(string)
        
        cv2.imshow("Imgae Crop",imgCrop)
    cv2.imshow("MAIN",img)
    cv2.waitKey(1)

2
23
23+
23+5
Enter 's' if you want to calculate or press any key to continue: s
28
